# Import Libraries & Dataset

In [ ]:
import pandas as pd
import pyspark

import os
import sys

In [ ]:
from pyspark.sql import SparkSession
# Create Spark session with Java security configuration
spark = SparkSession.builder.appName("CICIoT2023_filter").getOrCreate()

# Read the CSV files 
df = spark.read.csv("data/ciciot2023/*.csv", header=True, inferSchema=True)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/06 06:35:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/06 06:35:50 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: CICIoT2023/*.csv.
java.io.FileNotFoundException: File CICIoT2023/*.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMeta

In [6]:
spark

In [7]:
df.printSchema()
df.show(5)

25/11/06 06:36:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- flow_duration: double (nullable = true)
 |-- Header_Length: double (nullable = true)
 |-- Protocol Type: double (nullable = true)
 |-- Duration: double (nullable = true)
 |-- Rate: double (nullable = true)
 |-- Srate: double (nullable = true)
 |-- Drate: double (nullable = true)
 |-- fin_flag_number: double (nullable = true)
 |-- syn_flag_number: double (nullable = true)
 |-- rst_flag_number: double (nullable = true)
 |-- psh_flag_number: double (nullable = true)
 |-- ack_flag_number: double (nullable = true)
 |-- ece_flag_number: double (nullable = true)
 |-- cwr_flag_number: double (nullable = true)
 |-- ack_count: double (nullable = true)
 |-- syn_count: double (nullable = true)
 |-- fin_count: double (nullable = true)
 |-- urg_count: double (nullable = true)
 |-- rst_count: double (nullable = true)
 |-- HTTP: double (nullable = true)
 |-- HTTPS: double (nullable = true)
 |-- DNS: double (nullable = true)
 |-- Telnet: double (nullable = true)
 |-- SMTP: double (nullable = 

In [8]:
df.columns

['flow_duration',
 'Header_Length',
 'Protocol Type',
 'Duration',
 'Rate',
 'Srate',
 'Drate',
 'fin_flag_number',
 'syn_flag_number',
 'rst_flag_number',
 'psh_flag_number',
 'ack_flag_number',
 'ece_flag_number',
 'cwr_flag_number',
 'ack_count',
 'syn_count',
 'fin_count',
 'urg_count',
 'rst_count',
 'HTTP',
 'HTTPS',
 'DNS',
 'Telnet',
 'SMTP',
 'SSH',
 'IRC',
 'TCP',
 'UDP',
 'DHCP',
 'ARP',
 'ICMP',
 'IPv',
 'LLC',
 'Tot sum',
 'Min',
 'Max',
 'AVG',
 'Std',
 'Tot size',
 'IAT',
 'Number',
 'Magnitue',
 'Radius',
 'Covariance',
 'Variance',
 'Weight',
 'label']

In [10]:
df.describe().show()

+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+--------------------+------------------+------------------+-----------------+--------------------+-------------------+------------------+----------------+
|summary|     flow_duration|     Header_Length|    Protocol Type| 

# Preprocessing

In [11]:
category_dict = {
    'DDoS-ACK_Fragmentation' : 'DDoS',
    'DDoS-HTTP_Flood' : 'DDoS',
    'DDoS-ICMP_Flood': 'DDoS',
    'DDoS-PSHACK_Flood': 'DDoS',
    'DDoS-RSTFINFlood': 'DDoS',
    'DDoS-SYN_Flood': 'DDoS',
    'DDoS-SlowLoris': 'DDoS',
    'DDoS-SynonymousIP_Flood': 'DDoS',
    'DDoS-TCP_Flood': 'DDoS',
    'DDoS-UDP_Flood': 'DDoS',
    'DDoS-UDP_Fragmentation': 'DDoS',
    'DDoS-ICMP_Fragmentation' : 'DDoS',

    'DoS-HTTP_Flood' : 'DoS',
    'DoS-SYN_Flood' : 'DoS',
    'DoS-TCP_Flood' : 'DoS',
    'DoS-UDP_Flood' : 'DoS',

    'DictionaryBruteForce' : 'BruteForce',

    'MITM-ArpSpoofing' : 'Spoofing',
    'DNS_Spoofing' : 'Spoofing',

    'Recon-HostDiscovery' : 'Recon',
    'Recon-OSScan' : 'Recon',
    'Recon-PingSweep' : 'Recon',
    'Recon-PortScan' : 'Recon',
    'VulnerabilityScan' : 'Recon',

    'SqlInjection' : 'Web-based',
    'CommandInjection' : 'Web-based',
    'Backdoor_Malware' : 'Web-based',
    'Uploading_Attack' : 'Web-based',
    'XSS' : 'Web-based',
    'BrowserHijacking' : 'Web-based',

    'Mirai-greeth_flood' : 'Mirai',
    'Mirai-greip_flood' : 'Mirai',
    'Mirai-udpplain' : 'Mirai',

    'BenignTraffic' : 'Benign'
}

In [12]:
# map original labels to broader categories
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def map_category(label):
    return category_dict.get(label)

map_category_udf = udf(map_category, StringType())
df = df.withColumn('category', map_category_udf(df['label']))
df.show(5)

+-----------------+-------------+-------------+--------+------------------+------------------+-----+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------+---------+---------+---------+---------+----+-----+---+------+----+---+---+---+---+----+---+----+---+---+-------+----+----+----+---+--------+-------------------+------+------------------+------+----------+--------+------+--------------------+--------+
|    flow_duration|Header_Length|Protocol Type|Duration|              Rate|             Srate|Drate|fin_flag_number|syn_flag_number|rst_flag_number|psh_flag_number|ack_flag_number|ece_flag_number|cwr_flag_number|ack_count|syn_count|fin_count|urg_count|rst_count|HTTP|HTTPS|DNS|Telnet|SMTP|SSH|IRC|TCP|UDP|DHCP|ARP|ICMP|IPv|LLC|Tot sum| Min| Max| AVG|Std|Tot size|                IAT|Number|          Magnitue|Radius|Covariance|Variance|Weight|               label|category|
+-----------------+-------------+-------------+-----

In [ ]:
# filter to Brute Force,  Spoofing, Recon and Benign categories only
df = df.filter(df['category'].isin(['BruteForce', 'Spoofing', 'Recon', 'Benign']))
df.groupBy('category').count().show()

+----------+-------+
|  category|  count|
+----------+-------+
|BruteForce|  13064|
|    Benign|1098195|
|  Spoofing| 486504|
|     Recon| 354565|
+----------+-------+



In [14]:
# export filtered data to parquet
df.write.parquet('ciciot2023_filtered.parquet')

25/11/06 06:39:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/11/06 06:39:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/11/06 06:39:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/11/06 06:39:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/11/06 06:39:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/11/06 06:39:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
25/11/06 06:39:50 WARN MemoryManager: Total allocation exceeds 95.